# imports 

In [2]:
import boto3
import os
import yaml
import pandas as pd
import numpy as np
# ! pip install xgboost

# Test endpoint 


In [6]:
import boto3

# Initialize the SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime')

# Specify the endpoint name
endpoint_name = 'bits-deploy-new-staging'

# Prepare the input data in CSV format
input_data = "1,22,1,0,3,8.92,7.001,66,15.0013,1,4,0,0,0,1,0,0,0 "

# Make the request to the SageMaker endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv', 
    Body=input_data
)
# print(response)

# Parse the response
result = response['Body'].read().decode()

# Print the result
print("Prediction:", result)


Prediction: 0.6400858163833618



# stream and load model directly from s3

In [10]:
import boto3
import io
import tarfile
import tempfile
import xgboost as xgb

bucket_name = 'sagemaker-project-961ef69e-26d3-4ec4-b5bd-81c4008d512e'
model_file_key = 'siemens-poc/xgboost/output/pipelines-95pn9jz65osv-siemens-poc-pipeline-SdcQTIpnkj/output/model.tar.gz'




s3 = boto3.resource('s3')

model_file_stream = io.BytesIO()
s3.Bucket(bucket_name).download_fileobj(model_file_key, model_file_stream)

# Set the stream position to the start
model_file_stream.seek(0)

# Extract the model file from the tar.gz archive in memory
with tarfile.open(fileobj=model_file_stream, mode="r:gz") as tar:
    # Extract the model file to a temporary file
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        for member in tar.getmembers():
            if member.name == 'xgboost-model':  # Adjust the file name as needed
                tmp_file.write(tar.extractfile(member).read())
                tmp_file.flush()
                break

# Load the XGBoost model from the temporary file
model = xgb.Booster()
model.load_model(tmp_file.name)



In [11]:
model

In [14]:
# s3://sagemaker-us-east-1-644383320443/
import io
file_key  = "siemens-poc/pipeline-jobs/test/test_x.csv"
s3_client = boto3.client("s3")
response = s3_client.get_object(Bucket=config['s3-bucket']['bucket_name'], Key=file_key)
file_content = response['Body'].read()
X_test = pd.read_csv(io.BytesIO(file_content), encoding='utf-8')
X_test.head(3)

,season,hr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,yr,mnth,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed
0,1,22,1,0,3,8.92,7.0010,66.0,15.0013,1,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3,11,1,1,3,26.78,30.0020,70.0,11.0014,0,9,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,9,1,1,2,7.98,5.0012,81.0,19.0012,0,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
file_key  = "siemens-poc/pipeline-jobs/test/test_y.csv"
response = s3_client.get_object(Bucket=config['s3-bucket']['bucket_name'], Key=file_key)
file_content = response['Body'].read()
y_test = pd.read_csv(io.BytesIO(file_content), encoding='utf-8')
y_test.head(3)

,cnt
0,276
1,145
2,75


In [21]:
test_x = xgb.DMatrix(X_test.values)
test_y = y_test.to_numpy()


In [24]:
# Run predictions
predictions = np.array(model.predict(test_x), dtype=float).squeeze()
print(predictions)

[0.64008582 2.22400331 1.03455913 ... 0.64008582 2.13660216 0.06261845]


In [25]:
# Evaluate predictions
test_results = pd.concat([pd.Series(predictions, name="y_pred", index=X_test.index),X_test,],axis=1,)
test_results.head()


,y_pred,season,hr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,yr,mnth,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed
0,0.640086,1,22,1,0,3,8.92,7.0010,66.0,15.0013,1,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2.224003,3,11,1,1,3,26.78,30.0020,70.0,11.0014,0,9,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.034559,2,9,1,1,2,7.98,5.0012,81.0,19.0012,0,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.865599,1,1,1,0,1,7.98,5.0012,87.0,19.0012,0,6,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2.867956,3,12,1,1,3,22.08,24.0026,65.0,7.0015,0,10,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.metrics import mean_squared_error
test_rmse = mean_squared_error(test_y, test_results["y_pred"])
report_dict = {"regression_metric":{"test_rmse":{"value":test_rmse}}}
print(f"Test-rmse: {test_rmse:.2f}")

Test-rmse: 70624.57
